<div id="table-of-contents" role="doc-toc">
<h2>Table des matières</h2>
<div id="text-table-of-contents" role="doc-toc">
<ul>
<li><a href="#org05d9e17">1. Informations générales</a></li>
<li><a href="#org31cdbb0">2. Valeurs manquantes ou aberrantes</a>
<ul>
<li><a href="#orge687e42">2.1. Trouver</a></li>
<li><a href="#org761935f">2.2. Corriger</a></li>
</ul>
</li>
<li><a href="#org4fbb05f">3. Corrélations</a></li>
<li><a href="#org564e0cf">4. Modèles linéaires</a>
<ul>
<li><a href="#org365e84f">4.1. carat → price</a></li>
<li><a href="#org8c0f114">4.2. depth, table → price</a></li>
<li><a href="#orgde10be3">4.3. x, y, z → price</a></li>
</ul>
</li>
<li><a href="#org2ca707e">5. Conclusion</a></li>
</ul>
</div>
</div>


In [1]:
%pip install seaborn

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/7b/e5/83fcd7e9db036c179e0352bfcd20f81d728197a16f883e7b90307a88e65e/seaborn-0.13.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


On va étudier la corrélation et les modèles linéaires pour le jeu de
données *diamonds* fourni en exemple par la bibliothèque *Seaborn*.
On peut charger ce jeu de données de la façon suivante :


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
diamonds = pd.read_csv("./Data/seaborn-data/diamonds.csv")

**Note :** Si le chargement échoue à cause d'un problème de configuration de proxy, on peut télécharger <a href="https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv">le fichier</a> avec le navigateur.



<a id="org05d9e17"></a>

# Informations générales



Commencer par afficher les noms et types de colonnes, ainsi que la
consommation en mémoire de la `DataFrame`.


In [6]:
#%load hints/exo_corr_1-1.py
#TODO
# On regarde le types des colonnes
print(diamonds.dtypes)
# on regarde la consommation en mémoire
print(diamonds.info(memory_usage='deep'))


carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 12.2 MB
None



<a id="org31cdbb0"></a>

# Valeurs manquantes ou aberrantes




<a id="orge687e42"></a>

## Trouver



Chercher les valeurs manquantes ou aberrantes, par exemple des tailles
ou prix (`x`, `y` , `z` …) à 0.


In [8]:
#%load hints/exo_corr_2-1.py
#TODO

print(diamonds.isna().sum())
print(diamonds[diamonds['x']==0])
print(diamonds[diamonds['y']==0])
print(diamonds[diamonds['z']==0])

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64
       carat        cut color clarity  depth  table  price    x     y    z
11182   1.07      Ideal     F     SI2   61.6   56.0   4954  0.0  6.62  0.0
11963   1.00  Very Good     H     VS2   63.3   53.0   5139  0.0  0.00  0.0
15951   1.14       Fair     G     VS1   57.5   67.0   6381  0.0  0.00  0.0
24520   1.56      Ideal     G     VS2   62.2   54.0  12800  0.0  0.00  0.0
26243   1.20    Premium     D    VVS1   62.1   59.0  15686  0.0  0.00  0.0
27429   2.25    Premium     H     SI2   62.8   59.0  18034  0.0  0.00  0.0
49556   0.71       Good     F     SI2   64.1   60.0   2130  0.0  0.00  0.0
49557   0.71       Good     F     SI2   64.1   60.0   2130  0.0  0.00  0.0
       carat        cut color clarity  depth  table  price    x    y    z
11963   1.00  Very Good     H     VS2   63.3   53.0   5139  0.0  0.0  0.0
15951   1.14       Fair     G     


<a id="org761935f"></a>

## Corriger



Est-il préférable de remplacer ou supprimer les valeurs manquantes ?
Dans ce dernier cas, en supprimant des lignes ou des colonnes ?

Supprimer les lignes avec des valeurs manquantes.


In [12]:
#%load hints/exo_corr_3-1.py
#TODO
# Suppresion des valeurs manquantes de type NA
diamonds = diamonds.dropna()
diamonds_dropped = diamonds.dropna(inplace=False)
# Suppresion des valeurs = 0
diamonds = diamonds[(diamonds[['x','y','z']] != 0).all(axis=1)]

Vérfier ensuite qu'il n'y a plus de valeurs manquantes / aberrantes.


In [13]:
#%load hints/exo_corr_4-1.py
#TODO
print((diamonds[['x','y','z']] == 0).sum())

x    0
y    0
z    0
dtype: int64



<a id="org4fbb05f"></a>

# Corrélations



Calculer et afficher la corrélation entre prix et carats.


In [14]:
#%load hints/exo_corr_5-1.py
#TODO
# corrélation entre le prix et carats avec corr()

correlation = diamonds['price'].corr(diamonds['carat'])
print("Correlation between price and carat: ", correlation)


Correlation between price and carat:  0.9215920634723977



<a id="org564e0cf"></a>

# Modèles linéaires



Calculer et afficher le modèle linéaire du prix (`price`) avec les
variables explicatives suivantes :

-   `carat`

-   `depth` and `table`

-   `x`, `y` and `z`



<a id="org365e84f"></a>

## carat → price



In [15]:
#%load hints/exo_corr_6-1_fr-FR.py
#TODO

import statsmodels.api as sm

Y = diamonds['price']
X = diamonds['carat']

# Ajout d'une constante à X
X = sm.add_constant(X)

# Construction du modèle
model = sm.OLS(Y, X)

# Ajustement du modèle aux données
results = model.fit()

# Affichage du résumé
print(results.summary())



                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                 3.039e+05
Date:                Thu, 05 Oct 2023   Prob (F-statistic):               0.00
Time:                        16:24:43   Log-Likelihood:            -4.7253e+05
No. Observations:               53920   AIC:                         9.451e+05
Df Residuals:                   53918   BIC:                         9.451e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2255.7688     13.052   -172.828      0.0


<a id="org8c0f114"></a>

## depth, table → price



In [16]:
#%load hints/exo_corr_7-1_fr-FR.py
#TODO
import statsmodels.api as sm

Y = diamonds['price']
X = diamonds[['depth', 'table']]

# Ajout d'une constante à X
X = sm.add_constant(X)

# Construction du modèle
model = sm.OLS(Y, X)

# Ajustement du modèle aux données
results = model.fit()

# Affichage du résumé
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     465.8
Date:                Thu, 05 Oct 2023   Prob (F-statistic):          2.75e-201
Time:                        16:26:56   Log-Likelihood:            -5.2309e+05
No. Observations:               53920   AIC:                         1.046e+06
Df Residuals:                   53917   BIC:                         1.046e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.508e+04   1004.517    -15.011      0.0

x, y, z → price
<a id="orgde10be3"></a>

## x, y, z → price



In [17]:
#%load hints/exo_corr_8-1.py
#TODO
import statsmodels.api as sm

Y = diamonds['price']
X = diamonds[['x', 'y', 'z']]

# Ajout d'une constante à X
X = sm.add_constant(X)

# Construction du modèle
model = sm.OLS(Y, X)

# Ajustement du modèle aux données
results = model.fit()

# Affichage du résumé
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                 6.659e+04
Date:                Thu, 05 Oct 2023   Prob (F-statistic):               0.00
Time:                        16:27:45   Log-Likelihood:            -4.8180e+05
No. Observations:               53920   AIC:                         9.636e+05
Df Residuals:                   53916   BIC:                         9.636e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -1.42e+04     41.347   -343.419      0.0


<a id="org2ca707e"></a>

# Conclusion



Quelles autre combinaisons de variables explicatives seraient
pertinentes et pourquoi ?  Comment prendre en compte les variables
catégoriques ?


In [18]:
#%load hints/exo_corr_9-1.py
#TODO

diamonds = pd.get_dummies(diamonds, columns=['cut'])
